## FPL GAFFER TOOL CALLING
This notebook contains experiments with boilerplate tools, showing how FPL Gaffer would chose the tools to call based on prompts from a user during conversations.

In [ ]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import os
from typing import List, Literal
from pydantic import BaseModel, Field
from langchain.tools import Tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

In [ ]:
# TODO: Use boilerplate functions for tools, assess
# with an LLM to test the tools that would be called
# for different kinds of prompts.

# NOTE: Use proper documentation for the notebook.

In [4]:
# Tool input schemas
class NewsSearchInput(BaseModel):
    """Input schema for the news search tool."""
    query: str = Field(..., description="The search query for news search.")

class PlayerByPositionInput(BaseModel):
    """Input schema for the player by position tool."""
    position: Literal["GKP", "DEF", "MID", "FWD"] = Field(..., description="Position to search for. One of: GK, DEF, MID, FWD.")
    max_price: float = Field(15.0, description="Maximum player price to search for in millions.")


class PlayerDataInput(BaseModel):
    """Input schema for the player data tool."""
    player_names: List[str] = Field(..., description="List of player(s) names to fetch data for.")


class FixturesForRangeInput(BaseModel):
    """Input schema for the fixtures for range tool."""
    num_gameweeks: int = Field(..., description="Number of upcoming gameweeks to fetch fixtures for.")

class UserTeamInfoInput(BaseModel):
    """Input schema for the user team info tool."""
    manager_id: int = Field(..., description="The user's FPL manager ID.")
    gameweek: int = Field(..., description="The current gameweek number to fetch data for.")

In [11]:
# Boilerplate tool functions
def news_search_tool(query: str):
    print(f"[news_search_tool] query: {query}")

def get_user_team_info_tool(manager_id: int, gameweek: int):
    print(f"[get_user_team_info_tool] manager_id={manager_id}, gameweek={gameweek}")

def get_players_by_position_tool(position: Literal["GKP", "DEF", "MID", "FWD"], max_price: float):
    print(f"[get_players_by_position_tool] position={position}, max_price={max_price}")

def get_player_data_tool(player_names: List[str]):
    print(f"[get_player_data_tool] player_names={player_names}")

def get_fixtures_for_range_tool(num_gameweeks: int):
    print(f"[get_fixtures_for_range_tool] num_gameweeks={num_gameweeks}")

# Create tool list
tools = [
    Tool(
        name="news_search_tool",
        description="Search for FPL news, expert analysis, injury updates, press conference information, etc."
                    "Use this when you need information about player/team news, injury, expert opinions, or "
                    "general FPL updates.",
        func=news_search_tool,
        args_schema=NewsSearchInput
    ),
    Tool(
        name="get_user_team_info_tool",
        description="Get comprehensive information about a user's FPL team including squad, transfers, "
                    "and finances. Use this when you need information about the user's team, players, or "
                    "financial situation.",
        func=get_user_team_info_tool,
        args_schema=UserTeamInfoInput
    ),
    Tool(
        name="get_players_by_position_tool",
        description="Get players by position and max price. Use this when you need information for player "
                    "replacements or transfer suggestions based on position and budget.",
        func=get_players_by_position_tool,
        args_schema=PlayerByPositionInput
    ),
    Tool(
        name="get_player_data_tool",
        description="Get detailed player data including stats, form, and injuries. Use this when you need "
                    "information about specific players.",
        func=get_player_data_tool,
        args_schema=PlayerDataInput
    ),
    Tool(
        name="get_fixtures_for_range_tool",
        description="Get fixtures from the current gameweek to the next x gameweeks. Use this when you need "
                    "information about upcoming fixtures or planning for future gameweeks.",
        func=get_fixtures_for_range_tool,
        args_schema=FixturesForRangeInput
    )
]

In [ ]:
TOOLS_DESCRIPTION = "\n".join(
    f"{t.name}: {t.description}" for t in tools
)
print(TOOLS_DESCRIPTION)

news_search_tool: Search for FPL news, expert analysis, injury updates, press conference information, etc.Use this when you need information about player/team news, injury, expert opinions, or general FPL updates.
get_user_team_info_tool: Get comprehensive information about a user's FPL team including squad, transfers, and finances. Use this when you need information about the user's team, players, or financial situation.
get_players_by_position_tool: Get players by position and max price. Use this when you need information for player replacements or transfer suggestions based on position and budget.
get_player_data_tool: Get detailed player data including stats, form, and injuries. Use this when you need information about specific players.
get_fixtures_for_range_tool: Get fixtures from the current gameweek to the next x gameweeks. Use this when you need information about upcoming fixtures or planning for future gameweeks.


In [ ]:
TOOL_ANALYSIS_PROMPT = """
You are an FPL conversational assistant that needs to decide the tools to call to assist the user's query.

User's query: {user_message}

Available tools:
{tools_description}

Determine which tools to call and in what order.

Respond ONLY with a list of tools in the order they should be called. e.g ["tool1", "tool2"].
"""

In [ ]:
class ToolResponse(BaseModel):
    tools_to_call: List[str] = Field(..., description="The list of tools to call")

In [ ]:
# Initialize llm
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.3-70b-versatile",
    temperature=0.4
)

In [ ]:
def run_query(q: str):
    messages = [TOOL_ANALYSIS_PROMPT, HumanMessage(content=q)]
    result = llm.invoke(messages)


